In [ ]:
from data_processor import DataProcessor
from nuplan.common.actor_state.state_representation import Point2D
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_utils import ScenarioMapping
from nuplan.planning.scenario_builder.nuplan_db.nuplan_scenario_builder import NuPlanScenarioBuilder
from nuplan.planning.scenario_builder.scenario_filter import ScenarioFilter
from nuplan.planning.utils.multithreading.worker_parallel import SingleMachineParallelExecutor
from diffusion_planner.data_process.utils import (
get_scenario_map,
get_filter_parameters,
sampled_tracked_objects_to_tensor,
)

from diffusion_planner.data_process.roadblock_utils import (
  route_roadblock_correction
)


from diffusion_planner.data_process.agent_process import(sampled_tracked_objects_to_array_list,
                                                         sampled_static_objects_to_array_list,
                                                         agent_past_process,)
from diffusion_planner.data_process.map_process import(map_process,
                                                       get_neighbor_vector_set_map)
import numpy as np


In [ ]:
db_path = "/cailiu2/Diffusion-Planner/data/2021.10.21.14.43.30_veh-28_01244_01519.db" # single db file
db_path = "/share/data_cold/open_data/nuplan/data/cache/mini"  # mini db files

map_path = "/share/data_cold/open_data/nuplan/maps"
save_path = "/cailiu2/Diffusion-Planner/data/processed"
map_version = "nuplan-maps-v1.0"

save_processed_path = "/cailiu2/Diffusion-Planner/data/processed"
total_scenarios = 10
scenario_mapping = ScenarioMapping(scenario_map=get_scenario_map(), subsample_ratio_override=0.5)
builder = NuPlanScenarioBuilder(db_path, map_path, None, None, map_version, scenario_mapping = scenario_mapping)
worker = SingleMachineParallelExecutor(use_process_pool=True)
scenario_filter = ScenarioFilter(*get_filter_parameters(num_scenarios_per_type=30000,
                                                          limit_total_scenarios=total_scenarios))
scenarios = builder.get_scenarios(scenario_filter, worker)
del worker, builder, scenario_filter
processor = DataProcessor(scenarios)

In [5]:
"""
ego
"""
ego_state = processor.scenario.initial_ego_state
ego_coords = Point2D(ego_state.rear_axle.x, ego_state.rear_axle.y)
anchor_ego_state = np.array([ego_state.rear_axle.x, ego_state.rear_axle.y, ego_state.rear_axle.heading], dtype=np.float64)
print(ego_coords)
print(anchor_ego_state)


"""
neighbor
"""
# neighbor_agent_past, neighbor_agents_types = processor.get_neighbor_agents()
neighbors = processor.scenario.initial_tracked_objects
neighbors.tracked_objects
# print(len(neighbors.tracked_objects))
# print(type(neighbors.tracked_objects))

neighbor_past = list(processor.scenario.get_past_tracked_objects(iteration= 0, time_horizon=1.0))
print(type(neighbor_past))
print(type(neighbor_past[0]))
# for neighbor in neighbors:
tracked_objects, tracked_objects_types = sampled_tracked_objects_to_tensor(neighbors.tracked_objects)
tracked_objects_past, _ = sampled_tracked_objects_to_array_list(neighbor_past)
static_objects, static_objects_types =sampled_static_objects_to_array_list(neighbor_past[-1])
# print(type(neighbor_past))
# print(len(neighbor_past))
tracked_objects_list = []

for tracked_object in tracked_objects:
  tracked_objects_list.append(tracked_object)
print(tracked_objects_past[0].shape)
print(len(tracked_objects_past))
print(len(static_objects))

num_agents = 30
num_statics = 30
max_ped_bike = 10
_, neighbor_past, _, static_objects = agent_past_process(None, tracked_objects, tracked_objects_types, num_agents,static_objects, static_objects_types, \
                                                          num_statics, max_ped_bike, anchor_ego_state)
# print(len(tracked_objects_list))


Point2D(x=330957.49299200956, y=4691143.579406448)
[3.30957493e+05 4.69114358e+06 5.43869085e-01]
<class 'list'>
<class 'nuplan.planning.simulation.observation.observation_type.DetectionsTracks'>
(57, 8)
10
4


In [11]:
'''
Map
'''
map_api = processor.scenario.map_api
route_roadblocks_ids = processor.scenario.get_route_roadblock_ids()

route_roadblocks_ids =route_roadblock_correction(
                    ego_state, map_api, route_roadblocks_ids
                    )
traffic_light_data = processor.scenario.get_traffic_light_status_at_iteration(iteration= 0)
coords, traffic_light_data, speed_limit, lane_route =get_neighbor_vector_set_map(
            map_api, processor._map_features, ego_coords, processor._radius, traffic_light_data)
vector_map = map_process(route_roadblocks_ids, anchor_ego_state, coords, traffic_light_data, speed_limit, lane_route, processor._map_features,
                            processor._max_elements, processor._max_points)


In [25]:
data = {"neighbor_agents_past": neighbor_past[:, -21:],
        "ego_current_state": np.array([0., 0., 1. ,0.], dtype=np.float32), # ego centric x, y, cos, sin
        "static_objects": static_objects}
data.update(vector_map)
import torch
from diffusion_planner.data_process.utils import convert_to_model_inputs
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
data = convert_to_model_inputs(data, device)


In [26]:
import os

# 处理时间格式

save_path = "/cailiu2/Diffusion-Planner/data/processed"
# 生成文件名
file_name = f"{processor.scenario.log_name}_{processor.scenario.token}_{processor.scenario.scenario_type}.pt"

# 确保文件名合法（移除特殊字符）
file_name = file_name.replace(":", "_").replace("/", "_").replace(" ", "_")

# 指定存放路径
os.makedirs(save_path, exist_ok=True)  # 确保目录存在

# 完整文件路径
file_path = os.path.join(save_path, file_name)
torch.save(data, file_path)
